In [9]:
from fairseq.models.bart import BARTModel
import torch
from rouge import Rouge, FilesRouge
from tqdm import tqdm_notebook as tqdm
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("gpu num: ", n_gpu)

gpu num:  1


In [10]:
torch.__version__

'1.4.0+cu92'

In [3]:
bart = BARTModel.from_pretrained(
    'checkpoints_multi_view',
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path='../data'
)

here!


In [11]:
bart.eval()

BARTHubInterface(
  (model): BARTModel(
    (encoder): TransformerEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
        (1): TransformerEnc

In [5]:
# bart.to(device)

In [6]:
from tqdm import tqdm as tqdm

In [7]:
# with open('data/train3000.source', encoding='utf-8') as source:
#     s1 = source.readlines()
#     s1 = [l for l in s1 if l != ' | \n']

In [12]:
count = 1
bsz = 8
with open('data/1800train_100dev_100test.source_sent_trans_cons_label.source', encoding='utf-8') as source, open('data/1800train_100dev_100test.source_sent_c99_label.source', encoding='utf-8') as source2, open('./1800train_100dev_100test_out.txt', 'wt', encoding='utf-8') as fout:
    s1 = source.readlines()
    s2 = source2.readlines()

    s1 = [l.strip() + ' | \n' for l in s1 if l != ' | \n']
    s2 = [l.strip() + ' | \n' for l in s2 if l != ' | \n']
    
    slines = [s1[0].strip()]
    slines2 = [s2[0].strip()]
    
    for i in tqdm(range(1, len(s1))):
        if count % bsz == 0:
            with torch.no_grad():
                hypotheses_batch = bart.sample(slines, sentences2 = slines2, balance = True, beam=4, lenpen=2.0, max_len_b=100, min_len=5, no_repeat_ngram_size=3)
            for hypothesis in hypotheses_batch:
                fout.write(hypothesis + '\n')
                fout.flush()
            slines = []
            slines2 = []

        slines.append(s1[i].strip())
        slines2.append(s2[i].strip())

        count += 1

    if slines != []:

        hypotheses_batch = bart.sample(slines, sentences2 = slines2, balance = True, beam=4, lenpen=2.0, max_len_b=100, min_len=5, no_repeat_ngram_size=3)
       

        for hypothesis in hypotheses_batch:
            fout.write(hypothesis + '\n')
            fout.flush()

        

C:\Users\gorde\AppData\Local\Temp\ipykernel_24960\1458547165.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(1, len(s1))):


  0%|          | 0/1999 [00:00<?, ?it/s]

In [7]:
from rouge import Rouge, FilesRouge

In [15]:
# hyp_path = './test_best_multi_attn_best.hypo'
# ref_path = './fairseq_multi_view_code/data3/test_sent_trans_cons_label.target'
# hypothesis = []
# with open(hyp_path, 'r') as f:
#     lines = f.readlines()
#     for l in lines:
#         hypothesis.append(l[:-1])

# reference = []
# with open(ref_path, 'r') as f:
#     lines = f.readlines()
#     for l in lines:
#         reference.append(l[:-1])

# rouge = Rouge()
# print('Test', rouge.get_scores(hypothesis, reference, avg = True))

Test {'rouge-1': {'f': 0.49009429017411965, 'p': 0.5079810307154228, 'r': 0.5198850300191744}, 'rouge-2': {'f': 0.25332958515740567, 'p': 0.26215165975601734, 'r': 0.2721887911613676}, 'rouge-l': {'f': 0.47529740021449296, 'p': 0.49092217200422267, 'r': 0.4978984240638189}}
